<a href="https://colab.research.google.com/github/MohamadSuhail/Cyber-bullying-tweet-prediction/blob/main/Copy_of_Cyberbullying_tweet_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# stopwords module
nltk.download('stopwords')

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('wordnet')

In [ ]:
stopwords_list=nltk.corpus.stopwords.words('english')

In [ ]:
#loading the data
tweets = pd.read_csv('/content/drive/MyDrive/datasets/cyberbullying/cyberbullying_tweets.csv')

In [ ]:
#sample view of the data
tweets.head()

In [ ]:
#overview dataset
tweets.info()

There are 47k records in the dataset  
There are no null values in the dataset

In [ ]:
#statistical analysis of dataset
tweets.describe()

There are 46017 unique values and 1675 duplicate values

In [ ]:
#dropping the duplicates
tweets.drop_duplicates(keep='first',inplace=True)

In [ ]:
# type of categories
classes= tweets['cyberbullying_type'].unique()
print('lenght of categories:',len(tweets.columns))
print('classes:',classes)

In [ ]:
#classes distribution
tweets['cyberbullying_type'].value_counts(normalize=True).plot(kind='bar',color='green')
plt.title('Composition of classes')
plt.xlabel('classes')
plt.ylabel('percentages')
plt.show()

Dataset has balanced classes

In [ ]:
#renaming the columns
tweets = tweets.rename(columns={'tweet_text':'text','cyberbullying_type':'type'})

In [ ]:
#sample of different categories of bullying
tweets.type.unique()

In [ ]:
#sample tweets 
#gender
tweets[tweets.type=='gender'].values[:3]

In [ ]:
#religion
tweets[tweets.type=='religion'].values[5:8]

In [ ]:
#non-bullying
tweets[tweets.type=='not_cyberbullying'].values[:3]

## Text analysis

## Text preprocessing

In [ ]:
# Clean the data
def text_preprocess(text):
    # Remove HTML tags
    text = re.sub('<.*?>', '', text)
    
    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub('[^a-zA-Z]', ' ', text).lower()
    
    # Remove URLs, mentions, and hashtags from the text
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    
    # Tokenize the text
    words = nltk.word_tokenize(text)
    # print(words)
    # Remove stopwords
    words = [w for w in words if w not in stopwords_list]
    
    # Stem the words
    stemmer = PorterStemmer()
    words = [stemmer.stem(w) for w in words]
    # #lemmatization
    # lemmatizer = WordNetLemmatizer()
    # words = [lemmatizer.lemmatize(w) for w in words]
    
    # Join the words back into a string
    text = ' '.join(words)
    return text

In [ ]:
#applying text preprocessing on text column
tweets['text'] = tweets['text'].apply(text_preprocess)

In [ ]:
tweets['text'].values[:3]

In [ ]:
#label encoding the targets
# label_encoder object knows how to understand word labels.
label_encoder = LabelEncoder()
  
# Encode labels in column 'species'.
tweets['type']= label_encoder.fit_transform(tweets['type'])

In [ ]:
#seperating features and targets
X = tweets['text']
y = tweets['type']

In [ ]:
#train and test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
#Tfidf vectorizer
term_idf = TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,2),max_features=5000)
#transformed train reviews
term_idf_train = term_idf.fit_transform(X_train)
#transformed test reviews
term_idf_test = term_idf.transform(X_test)
print('Tfidf_train:',term_idf_train.shape)
print('Tfidf_test:',term_idf_test.shape)

In [ ]:
#countvectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
#transformed train reviews
count_vect_train = count_vect.fit_transform(X_train)
#transformed test reviews
count_vect_test = count_vect.transform(X_test)
print('Tfidf_train:',count_vect_train.shape)
print('Tfidf_test:',count_vect_test.shape)

## Machine learning models

##SVM

In [ ]:
from sklearn.svm import SVC
svm_model_linear = SVC(kernel= 'rbf',C=1).fit(count_vect_train, y_train)

In [ ]:

svm_predictions  = svm_model_linear.predict(count_vect_test)
svm_accuracy = svm_model_linear.score(count_vect_test, y_test)

In [ ]:
svm_accuracy

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#classification report
print(classification_report(y_test,svm_predictions))

In [ ]:
#confusion matrix
cm = confusion_matrix(y_test,svm_predictions)
sns.heatmap(cm,annot=True,fmt='g')
plt.show()

##Multinomial Naive Bayes

In [ ]:
mnb = MultinomialNB(force_alpha=True,fit_prior=True)
mnb.fit(count_vect_train, y_train)

In [ ]:
mnb_predictions  = mnb.predict(count_vect_test)
mnb_accuracy = mnb.score(count_vect_test, y_test)

In [ ]:
mnb_accuracy

In [ ]:
print(classification_report(y_test,mnb_predictions))

In [ ]:
cm = confusion_matrix(y_test,mnb_predictions)
sns.heatmap(cm,annot=True,fmt='g')
plt.show()

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
lr.fit(count_vect_train, y_train)

In [ ]:
lr_predictions  = lr.predict(count_vect_test)
accuracy = lr.score(count_vect_test, y_test)

In [ ]:
print(classification_report(y_test,lr_predictions))

In [ ]:
cm = confusion_matrix(y_test,lr_predictions)
sns.heatmap(cm,annot=True,fmt='g')
plt.show()

## RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(count_vect_train, y_train)

In [ ]:
rf_predictions  = rf.predict(count_vect_test)
accuracy_rf = rf.score(count_vect_test, y_test)

In [ ]:
accuracy_rf

In [ ]:
print(classification_report(y_test,rf_predictions))

In [ ]:
cm = confusion_matrix(y_test,rf_predictions)
sns.heatmap(cm,annot=True,fmt='g')
plt.show()

In [ ]:
#results analysis
model_accu = [['SVM', 10], ['LogisticRegression', 15], ['RandomForest', 14],['MultinomialNaiveBayes',]]
pd.DataFrame()